## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-18-16-51-00 +0000,wsj,Opinion | Take a Nonconformist Out to Lunch,https://www.wsj.com/opinion/take-a-nonconformi...
1,2026-01-18-16-39-49 +0000,bbc,Chile declares 'state of catastrophe' as deadl...,https://www.bbc.com/news/articles/cz7ypw4rjrzo...
2,2026-01-18-16-38-00 +0000,wsj,"Pentagon places 1,500 soldiers on alert for po...",https://www.wsj.com/politics/policy/pentagon-p...
3,2026-01-18-16-31-54 +0000,nyt,Syrian Government and Kurdish-led Force Agree ...,https://www.nytimes.com/2026/01/18/world/middl...
4,2026-01-18-16-28-58 +0000,nyt,"Pentagon Tells 1,500 Troops to Prepare for Pos...",https://www.nytimes.com/2026/01/18/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2356/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,37
30,greenland,13
225,ice,12
15,minnesota,10
137,iran,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
77,2026-01-18-08-00-32 +0000,wapo,Melting ice may raise Greenland’s value. Trump...,https://www.washingtonpost.com/climate-environ...,79
89,2026-01-18-06-01-39 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...,74
82,2026-01-18-07-21-34 +0000,bbc,Trump tariff threat over Greenland 'unacceptab...,https://www.bbc.com/news/articles/cy4qjwk9n2no...,68
72,2026-01-18-10-00-00 +0000,wsj,Americans are split on President Trump’s opera...,https://www.wsj.com/politics/policy/voters-nar...,67
133,2026-01-17-23-39-58 +0000,wapo,Trump announces tariffs on European countries ...,https://www.washingtonpost.com/politics/2026/0...,67


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
77,79,2026-01-18-08-00-32 +0000,wapo,Melting ice may raise Greenland’s value. Trump...,https://www.washingtonpost.com/climate-environ...
123,33,2026-01-18-00-41-25 +0000,nypost,Minnesota National Guard on ‘standby’ and reve...,https://nypost.com/2026/01/17/us-news/minnesot...
174,32,2026-01-17-18-31-56 +0000,nypost,Venezuelan interior minister — a Nicolas Madur...,https://nypost.com/2026/01/17/world-news/venez...
138,30,2026-01-17-23-08-39 +0000,bbc,'Young 40s': Gen Z has found a new way to mock...,https://www.bbc.com/news/articles/c78vw30p1d2o...
142,28,2026-01-17-22-36-41 +0000,nypost,US strike kills Al Qaeda leader with direct ti...,https://nypost.com/2026/01/17/world-news/us-ki...
161,27,2026-01-17-20-20-55 +0000,nyt,World Leaders Consider Joining Trump’s ‘Board ...,https://www.nytimes.com/2026/01/17/world/middl...
34,24,2026-01-18-13-00-00 +0000,startribune,Wondering what to do indoors with kids on a co...,https://www.startribune.com/best-indoor-playgr...
162,24,2026-01-17-20-14-57 +0000,bbc,Iran supreme leader acknowledges thousands kil...,https://www.bbc.com/news/articles/ckglee733wno...
37,23,2026-01-18-12-00-00 +0000,startribune,‘We don’t want to be like Minneapolis’: Feelin...,https://www.startribune.com/discord-over-immig...
72,22,2026-01-18-10-00-00 +0000,wsj,Americans are split on President Trump’s opera...,https://www.wsj.com/politics/policy/voters-nar...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
